<a href="https://colab.research.google.com/github/gabrielcarcedo/SargazoClassification_ViT/blob/main/MeIA_Sargazo_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow import keras
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/MeIA Sargazo Dataset/

/content/drive/MyDrive/MeIA Sargazo Dataset


In [ ]:
!ls

images	labels


# Funciones de aumento de datos

In [ ]:
def flip_aug(image):
    return tf.image.flip_left_right(image)
def saturation_aug(image, value=1.5):
    return tf.image.adjust_saturation(image, value)
def brightness_aug(image, value=0.25):
    return tf.image.adjust_brightness(image, value)
def hue_aug(image, value=0.2):
    return tf.image.adjust_hue(image, value)
def contrast_aug(image, value=1):
    return tf.image.adjust_contrast(image, value)
def rotation_aug(image, angle=np.random.randint(-15, 15)):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)
    return rotated

def visualize(original, augmented):
    fig = plt.figure()
    plt.subplot(1,2,1)
    plt.title('Original')
    plt.imshow(original)
    plt.axis("off")

    plt.subplot(1,2,2)
    plt.title('Aumentada')
    plt.imshow(augmented)
    plt.axis("off")
    plt.show()

## Labels de entranamiento

In [ ]:
df_train = pd.read_csv('labels/labels.csv')
df_train.head()

,image_name,place,date,label,scene
0,i0001.jpg,Xpu-Ha,10/7/2022,nada,playa
1,i0002.jpg,Xpu-Ha,10/7/2022,nada,playa
2,i0003.jpg,Xpu-Ha,10/7/2022,bajo,playa
3,i0004.jpg,Xpu-Ha,10/7/2022,bajo,playa
4,i0005.jpg,Xpu-Ha,10/3/2022,bajo,playa


In [ ]:
df_train.shape

(2226, 5)

## Labels de test

In [ ]:
df_test = pd.read_csv('labels/test.csv')
df_test.head()

,image_name,place,date
0,s0826.jpg,Akumal,8/4/2019
1,s0827.jpg,Akumal,8/4/2019
2,s0828.jpg,Playa del Carmen,8/3/2019
3,s0829.jpg,Mahahual,8/3/2019
4,s0830.jpg,Punta Venado Riviera Maya,8/3/2019


In [ ]:
df_test.shape

(150, 3)

## Lista de imágenes

In [ ]:
images_list = os.listdir('images')
images_list.sort()

In [ ]:
len(images_list)

2376

## Obtener los tamaños máximos para hacer resize con padding

In [ ]:
resized_img_path = 'resized_images'
os.mkdir(resized_img_path)

In [ ]:
image_size = 224
for image in images_list:
    img = cv2.imread('images/' + image)
    h, w = img.shape[:2]

    # Factor de escala
    scale = image_size / max(h, w)

    # Resize
    img = cv2.resize(img, (int(w * scale), int(h * scale)))

    # Padding
    h_resized, w_resized = img.shape[:2]
    pad_top = (image_size - h_resized) // 2
    pad_bottom = image_size - h_resized - pad_top
    pad_left = (image_size - w_resized) // 2
    pad_right = image_size - w_resized - pad_left

    # Guardar imagen
    img = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    cv2.imwrite(resized_img_path + '/' + image, img)
    print(image)

i0001.jpg
i0002.jpg
i0003.jpg
i0004.jpg
i0005.jpg
i0006.jpg
i0007.jpg
i0008.jpg
i0009.jpg
i0010.jpg
i0011.jpg
i0012.jpg
i0013.jpg
i0014.jpg
i0015.jpg
i0016.jpg
i0017.jpg
i0018.jpg
i0019.jpg
i0020.jpg
i0021.jpg
i0022.jpg
i0023.jpg
i0024.jpg
i0025.jpg
i0026.jpg
i0028.jpg
i0029.jpg
i0030.jpg
i0031.jpg
i0032.jpg
i0033.jpg
i0034.jpg
i0035.jpg
i0036.jpg
i0037.jpg
i0038.jpg
i0039.jpg
i0040.jpg
i0041.jpg
i0042.jpg
i0043.jpg
i0044.jpg
i0045.jpg
i0046.jpg
i0047.jpg
i0048.jpg
i0049.jpg
i0050.jpg
i0051.jpg
i0052.jpg
i0053.jpg
i0054.jpg
i0055.jpg
i0056.jpg
i0057.jpg
i0058.jpg
i0060.jpg
i0061.jpg
i0062.jpg
i0063.jpg
i0064.jpg
i0065.jpg
i0066.jpg
i0067.jpg
i0068.jpg
i0069.jpg
i0070.jpg
i0071.jpg
i0072.jpg
i0073.jpg
i0074.jpg
i0075.jpg
i0076.jpg
i0077.jpg
i0078.jpg
i0079.jpg
i0080.jpg
i0083.jpg
i0085.jpg
i0086.jpg
i0087.jpg
i0090.jpg
i0091.jpg
i0092.jpg
i0093.jpg
i0094.jpg
i0095.jpg
i0096.jpg
i0097.jpg
i0098.jpg
i0099.jpg
i0100.jpg
i0101.jpg
i0102.jpg
i0103.jpg
i0104.jpg
i0105.jpg
i0106.jpg
i0107.jpg


In [ ]:
images_train_list = df_train['image_name'].tolist()
images_test_list = df_test['image_name'].tolist()
print(len(images_train_list))
print(len(images_test_list))

2226
150


In [ ]:
df_train['label'].unique()

array(['nada', 'bajo', 'moderado', 'abundante', 'excesivo'], dtype=object)

In [ ]:
label_dict = {'nada':0, 'bajo':1, 'moderado':2, 'abundante':3, 'excesivo':4}
df_train['label_num'] = df_train['label'].map(label_dict)
df_train.head()

,image_name,place,date,label,scene,label_num
0,i0001.jpg,Xpu-Ha,10/7/2022,nada,playa,0
1,i0002.jpg,Xpu-Ha,10/7/2022,nada,playa,0
2,i0003.jpg,Xpu-Ha,10/7/2022,bajo,playa,1
3,i0004.jpg,Xpu-Ha,10/7/2022,bajo,playa,1
4,i0005.jpg,Xpu-Ha,10/3/2022,bajo,playa,1


In [ ]:
images_train_nada = df_train[df_train['label'] == 'nada']['image_name'].tolist()
images_train_bajo = df_train[df_train['label'] == 'bajo']['image_name'].tolist()
images_train_moderado = df_train[df_train['label'] == 'moderado']['image_name'].tolist()
images_train_abundante = df_train[df_train['label'] == 'abundante']['image_name'].tolist()
images_train_excesivo = df_train[df_train['label'] == 'excesivo']['image_name'].tolist()
print(len(images_train_nada))
print(len(images_train_bajo))
print(len(images_train_moderado))
print(len(images_train_abundante))
print(len(images_train_excesivo))

1041
664
236
106
179


# Aumentos para la clase bajo

In [ ]:
for image in images_train_bajo:
    img = cv2.imread('resized_images/' + image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    augmented = flip_aug(img)
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_flip.jpg', cv2.cvtColor(augmented.numpy(), cv2.COLOR_RGB2BGR))
    original_registro = df_train[df_train['image_name'] == image]
    augmented_registro = original_registro.copy()
    augmented_registro['image_name'] = image.split('.')[0] + '_flip.jpg'
    df_train = pd.concat([df_train, augmented_registro], ignore_index=True)
    print(image)

i0003.jpg
i0004.jpg
i0005.jpg
i0006.jpg
i0007.jpg
i0008.jpg
i0014.jpg
i0015.jpg
i0021.jpg
i0022.jpg
i0031.jpg
i0033.jpg
i0034.jpg
i0040.jpg
i0043.jpg
i0044.jpg
i0045.jpg
i0052.jpg
i0054.jpg
i0058.jpg
i0061.jpg
i0062.jpg
i0080.jpg
i0085.jpg
i0109.jpg
i0111.jpg
i0112.jpg
i0113.jpg
i0131.jpg
i0133.jpg
i0136.jpg
i0137.jpg
i0138.jpg
i0139.jpg
i0140.jpg
i0141.jpg
i0142.jpg
i0143.jpg
i0144.jpg
i0150.jpg
i0167.jpg
i0168.jpg
i0169.jpg
i0170.jpg
i0171.jpg
i0172.jpg
i0173.jpg
i0174.jpg
i0175.jpg
i0176.jpg
i0179.jpg
i0180.jpg
i0181.jpg
i0182.jpg
i0183.jpg
i0184.jpg
i0185.jpg
i0187.jpg
i0188.jpg
i0189.jpg
i0198.jpg
i0199.jpg
j0001.jpg
k0012.jpg
k0013.jpg
k0015.jpg
k0018.jpg
k0022.jpg
k0023.jpg
k0050.jpg
k0051.jpg
k0055.jpg
k0056.jpg
k0064.jpg
k0110.jpg
k0154.jpg
k0165.jpg
k0181.jpg
k0182.jpg
k0184.jpg
k0185.jpg
k0186.jpg
k0189.jpg
k0193.jpg
k0197.jpg
k0199.jpg
k0235.jpg
k0236.jpg
k0239.jpg
k0241.jpg
k0242.jpg
k0243.jpg
k0244.jpg
k0247.jpg
k0265.jpg
k0266.jpg
k0267.jpg
k0270.jpg
k0287.jpg
k0289.jpg


# Aumentos para la clase moderado

In [ ]:
for image in images_train_moderado:
    img = cv2.imread('resized_images/' + image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    augmented1 = flip_aug(img)
    augmented2 = saturation_aug(img)
    augmented3 = brightness_aug(img)
    augmented4 = contrast_aug(img)
    augmented5 = rotation_aug(img)

    cv2.imwrite('resized_images/' + image.split('.')[0] + '_flip.jpg', cv2.cvtColor(augmented1.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_saturation.jpg', cv2.cvtColor(augmented2.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_brightness.jpg', cv2.cvtColor(augmented3.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_contrast.jpg', cv2.cvtColor(augmented4.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_rotation.jpg', cv2.cvtColor(augmented5, cv2.COLOR_RGB2BGR))

    original_registro = df_train[df_train['image_name'] == image]
    augmented1_registro = original_registro.copy()
    augmented1_registro['image_name'] = image.split('.')[0] + '_flip.jpg'
    augmented2_registro = original_registro.copy()
    augmented2_registro['image_name'] = image.split('.')[0] + '_saturation.jpg'
    augmented3_registro = original_registro.copy()
    augmented3_registro['image_name'] = image.split('.')[0] + '_brightness.jpg'
    augmented4_registro = original_registro.copy()
    augmented4_registro['image_name'] = image.split('.')[0] + '_contrast.jpg'
    augmented5_registro = original_registro.copy()
    augmented5_registro['image_name'] = image.split('.')[0] + '_rotation.jpg'
    df_train = pd.concat([df_train, augmented1_registro, augmented2_registro, augmented3_registro, augmented4_registro, augmented5_registro], ignore_index=True)
    print(image)

i0032.jpg
i0035.jpg
i0036.jpg
i0037.jpg
i0038.jpg
i0039.jpg
i0041.jpg
i0046.jpg
i0050.jpg
i0055.jpg
i0060.jpg
i0071.jpg
i0072.jpg
i0073.jpg
i0074.jpg
i0075.jpg
i0076.jpg
i0077.jpg
i0078.jpg
i0079.jpg
i0083.jpg
i0092.jpg
i0093.jpg
i0097.jpg
i0114.jpg
i0178.jpg
i0186.jpg
k0017.jpg
k0025.jpg
k0039.jpg
k0040.jpg
k0041.jpg
k0067.jpg
k0070.jpg
k0071.jpg
k0104.jpg
k0310.jpg
k0341.jpg
k0342.jpg
k0345.jpg
k0347.jpg
k0348.jpg
k0354.jpg
k0356.jpg
k0359.jpg
k0360.jpg
k0361.jpg
k0405.jpg
k0406.jpg
k0407.jpg
k0410.jpg
k0413.jpg
k0416.jpg
k0418.jpg
k0485.jpg
k0486.jpg
k0497.jpg
k0510.jpg
k0539.jpg
k0584.jpg
k0608.jpg
k0609.jpg
k0619.jpg
k0621.jpg
k0622.jpg
k0624.jpg
k0659.jpg
k0660.jpg
k0679.jpg
k0817.jpg
k0984.jpg
k0985.jpg
r027.jpg
r035.jpg
r037.jpg
r040.jpg
r046.jpg
r047.jpg
r053.jpg
r054.jpg
r060.jpg
r063.jpg
r070.jpg
r082.jpg
r102.jpg
r109.jpg
r139.jpg
r161.jpg
r171.jpg
r174.jpg
r179.jpg
r181.jpg
r195.jpg
r202.jpg
r208.jpg
r210.jpg
r217.jpg
r225.jpg
r233.jpg
r236.jpg
r257.jpg
r258.jpg
r282.jpg
r

# Aumentos para la clase abundante

In [ ]:
for image in images_train_abundante:
    img = cv2.imread('resized_images/' + image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    augmented1 = flip_aug(img)
    augmented2 = saturation_aug(img)
    augmented3 = brightness_aug(img)
    augmented4 = contrast_aug(img)
    augmented5 = rotation_aug(img)
    augmented6 = hue_aug(img)
    augmented7 = rotation_aug(img)
    augmented8 = saturation_aug(augmented1)
    augmented9 = brightness_aug(augmented1)
    augmented10 = contrast_aug(augmented1)

    cv2.imwrite('resized_images/' + image.split('.')[0] + '_flip.jpg', cv2.cvtColor(augmented1.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_saturation.jpg', cv2.cvtColor(augmented2.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_brightness.jpg', cv2.cvtColor(augmented3.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_contrast.jpg', cv2.cvtColor(augmented4.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_rotation.jpg', cv2.cvtColor(augmented5, cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_hue.jpg', cv2.cvtColor(augmented6.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_rotation2.jpg', cv2.cvtColor(augmented7, cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_saturation2.jpg', cv2.cvtColor(augmented8.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_brightness2.jpg', cv2.cvtColor(augmented9.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_contrast2.jpg', cv2.cvtColor(augmented10.numpy(), cv2.COLOR_RGB2BGR))

    original_registro = df_train[df_train['image_name'] == image]
    augmented1_registro = original_registro.copy()
    augmented1_registro['image_name'] = image.split('.')[0] + '_flip.jpg'
    augmented2_registro = original_registro.copy()
    augmented2_registro['image_name'] = image.split('.')[0] + '_saturation.jpg'
    augmented3_registro = original_registro.copy()
    augmented3_registro['image_name'] = image.split('.')[0] + '_brightness.jpg'
    augmented4_registro = original_registro.copy()
    augmented4_registro['image_name'] = image.split('.')[0] + '_contrast.jpg'
    augmented5_registro = original_registro.copy()
    augmented5_registro['image_name'] = image.split('.')[0] + '_rotation.jpg'
    augmented6_registro = original_registro.copy()
    augmented6_registro['image_name'] = image.split('.')[0] + '_hue.jpg'
    augmented7_registro = original_registro.copy()
    augmented7_registro['image_name'] = image.split('.')[0] + '_rotation2.jpg'
    augmented8_registro = original_registro.copy()
    augmented8_registro['image_name'] = image.split('.')[0] + '_saturation2.jpg'
    augmented9_registro = original_registro.copy()
    augmented9_registro['image_name'] = image.split('.')[0] + '_brightness2.jpg'
    augmented10_registro = original_registro.copy()
    augmented10_registro['image_name'] = image.split('.')[0] + '_contrast2.jpg'

    df_train = pd.concat([df_train, augmented1_registro, augmented2_registro, augmented3_registro, augmented4_registro, augmented5_registro, augmented6_registro, augmented7_registro, augmented8_registro, augmented9_registro, augmented10_registro], ignore_index=True)
    print(image)

i0042.jpg
i0047.jpg
i0048.jpg
i0049.jpg
i0056.jpg
i0057.jpg
i0063.jpg
i0064.jpg
i0065.jpg
i0066.jpg
i0086.jpg
i0118.jpg
i0119.jpg
i0120.jpg
i0121.jpg
i0122.jpg
i0124.jpg
i0126.jpg
i0129.jpg
i0165.jpg
i0166.jpg
i0194.jpg
j0002.jpg
j0004.jpg
j0005.jpg
j0416.jpg
j0417.jpg
j0419.jpg
k0021.jpg
k0024.jpg
k0026.jpg
k0027.jpg
k0369.jpg
k0371.jpg
k0372.jpg
k0374.jpg
k0375.jpg
k0376.jpg
k0377.jpg
k0385.jpg
k0386.jpg
k0393.jpg
k0394.jpg
k0395.jpg
k0396.jpg
k0398.jpg
k0484.jpg
k0499.jpg
k0500.jpg
k0537.jpg
k0538.jpg
r013.jpg
r016.jpg
r017.jpg
r036.jpg
r038.jpg
r055.jpg
r106.jpg
r266.jpg
r267.jpg
r331.jpg
r372.jpg
r373.jpg
r393.jpg
r408.jpg
r409.jpg
r421.jpg
r422.jpg
r429.jpg
r440.jpg
r475.jpg
s0150.jpg
s0173.jpg
s0174.jpg
s0230.jpg
s0232.jpg
s0233.jpg
s0234.jpg
s0240.jpg
s0255.jpg
s0260.jpg
s0275.jpg
s0310.jpg
s0334.jpg
s0384.jpg
s0385.jpg
s0388.jpg
s0418.jpg
s0483.jpg
s0546.jpg
s0548.jpg
s0549.jpg
s0551.jpg
s0552.jpg
s0563.jpg
s0575.jpg
s0631.jpg
s0654.jpg
s0669.jpg
s0670.jpg
s0680.jpg
s0780.jpg


# Aumentos para la clase excesivo

In [ ]:
for image in images_train_excesivo:
    img = cv2.imread('resized_images/' + image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    augmented1 = flip_aug(img)
    augmented2 = saturation_aug(img)
    augmented3 = brightness_aug(img)
    augmented4 = contrast_aug(img)
    augmented5 = rotation_aug(img)
    augmented6 = hue_aug(img)

    cv2.imwrite('resized_images/' + image.split('.')[0] + '_flip.jpg', cv2.cvtColor(augmented1.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_saturation.jpg', cv2.cvtColor(augmented2.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_brightness.jpg', cv2.cvtColor(augmented3.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_contrast.jpg', cv2.cvtColor(augmented4.numpy(), cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_rotation.jpg', cv2.cvtColor(augmented5, cv2.COLOR_RGB2BGR))
    cv2.imwrite('resized_images/' + image.split('.')[0] + '_hue.jpg', cv2.cvtColor(augmented6.numpy(), cv2.COLOR_RGB2BGR))

    original_registro = df_train[df_train['image_name'] == image]
    augmented1_registro = original_registro.copy()
    augmented1_registro['image_name'] = image.split('.')[0] + '_flip.jpg'
    augmented2_registro = original_registro.copy()
    augmented2_registro['image_name'] = image.split('.')[0] + '_saturation.jpg'
    augmented3_registro = original_registro.copy()
    augmented3_registro['image_name'] = image.split('.')[0] + '_brightness.jpg'
    augmented4_registro = original_registro.copy()
    augmented4_registro['image_name'] = image.split('.')[0] + '_contrast.jpg'
    augmented5_registro = original_registro.copy()
    augmented5_registro['image_name'] = image.split('.')[0] + '_rotation.jpg'
    augmented6_registro = original_registro.copy()
    augmented6_registro['image_name'] = image.split('.')[0] + '_hue.jpg'

    df_train = pd.concat([df_train, augmented1_registro, augmented2_registro, augmented3_registro, augmented4_registro, augmented5_registro, augmented6_registro], ignore_index=True)
    print(image)

i0098.jpg
i0099.jpg
i0100.jpg
i0101.jpg
i0102.jpg
i0103.jpg
i0104.jpg
i0105.jpg
i0106.jpg
i0107.jpg
i0108.jpg
i0116.jpg
i0117.jpg
i0145.jpg
i0147.jpg
i0148.jpg
i0191.jpg
i0192.jpg
i0193.jpg
i0195.jpg
i0196.jpg
j0003.jpg
k0046.jpg
k0048.jpg
k0373.jpg
k0384.jpg
k0446.jpg
k0820.jpg
r001.jpg
r002.jpg
r003.jpg
r007.jpg
r008.jpg
r009.jpg
r010.jpg
r011.jpg
r012.jpg
r014.jpg
r022.jpg
r023.jpg
r024.jpg
r025.jpg
r029.jpg
r030.jpg
r034.jpg
r039.jpg
r042.jpg
r043.jpg
r044.jpg
r045.jpg
r048.jpg
r049.jpg
r050.jpg
r051.jpg
r052.jpg
r056.jpg
r059.jpg
r064.jpg
r065.jpg
r066.jpg
r072.jpg
r076.jpg
r083.jpg
r084.jpg
r086.jpg
r087.jpg
r088.jpg
r092.jpg
r097.jpg
r098.jpg
r103.jpg
r110.jpg
r115.jpg
r117.jpg
r127.jpg
r128.jpg
r130.jpg
r133.jpg
r134.jpg
r135.jpg
r136.jpg
r138.jpg
r140.jpg
r141.jpg
r142.jpg
r143.jpg
r144.jpg
r146.jpg
r147.jpg
r158.jpg
r159.jpg
r170.jpg
r186.jpg
r187.jpg
r201.jpg
r206.jpg
r207.jpg
r209.jpg
r211.jpg
r213.jpg
r215.jpg
r216.jpg
r218.jpg
r220.jpg
r221.jpg
r234.jpg
r247.jpg
r254.jpg


In [ ]:
df_train.shape

(6204, 6)

In [ ]:
images_train_nada = df_train[df_train['label'] == 'nada']['image_name'].tolist()
images_train_bajo = df_train[df_train['label'] == 'bajo']['image_name'].tolist()
images_train_moderado = df_train[df_train['label'] == 'moderado']['image_name'].tolist()
images_train_abundante = df_train[df_train['label'] == 'abundante']['image_name'].tolist()
images_train_excesivo = df_train[df_train['label'] == 'excesivo']['image_name'].tolist()
print(len(images_train_nada))
print(len(images_train_bajo))
print(len(images_train_moderado))
print(len(images_train_abundante))
print(len(images_train_excesivo))

1041
1328
1416
1166
1253


In [ ]:
images_list = os.listdir('resized_images')
images_list.sort()
print(len(images_list))

6354


In [ ]:
csv_path = 'labels/labels_augmented.csv'
df_train.to_csv(csv_path, index=False)
